<a href="https://colab.research.google.com/github/oluwafemidiakhoa/AIreasearcher/blob/main/Climatesheat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install transformers datasets torch openai
!pip install  kaggle



In [22]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
##from transformers import VisionTransformerForImageClassification, ViTFeatureExtractor
from datasets import load_dataset
import requests

import openai
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import os
import openai  # OpenAI's Python library
from kaggle.api.kaggle_api_extended import KaggleApi
from google.colab import userdata




API clients initialized successfully.


In [25]:
# Importing necessary libraries
import os
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import openai
from kaggle.api.kaggle_api_extended import KaggleApi
import folium
import pandas as pd
import numpy as np
from datetime import datetime
import json

# Set environment variables for OpenAI and Kaggle credentials
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")
os.environ["HF_TOKEN"]= userdata.get("HF_TOKEN")


# Initialize OpenAI client
client = openai.Client(api_key=os.environ.get("OPENAI_API_KEY"))

# Initialize Kaggle API client
api = KaggleApi()
api.authenticate()

print("API clients initialized successfully.")

# Step 1: Download and Load Climate-Related Datasets from Kaggle
def download_kaggle_dataset(dataset_slug, dataset_path):
    """Download a dataset from Kaggle."""
    api.dataset_download_files(dataset_slug, path=dataset_path, unzip=True)
    print(f"Dataset downloaded to {dataset_path}.")

# Example Kaggle dataset: Climate Change Earth Surface Temperature Data
download_kaggle_dataset("berkeleyearth/climate-change-earth-surface-temperature-data", "./data")
climate_data = pd.read_csv("./data/GlobalTemperatures.csv")

# Step 2: Preprocess the Dataset
def preprocess_climate_data(df):
    """Preprocess climate data for analysis."""
    df["dt"] = pd.to_datetime(df["dt"])
    df["Year"] = df["dt"].dt.year
    df = df.groupby("Year").mean().reset_index()
    return df

climate_data = preprocess_climate_data(climate_data)

# Step 3: Train a Climate Insight Model using Hugging Face
def train_nlp_model():
    """Train or fine-tune an NLP model for climate text analysis."""
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    climate_texts = ["Global warming is causing ice caps to melt.", "The carbon emissions are reducing due to policy changes."]
    labels = [1, 0]  # 1: Problematic, 0: Positive

    inputs = tokenizer(climate_texts, padding=True, truncation=True, return_tensors="pt")
    labels = torch.tensor(labels).unsqueeze(0)

    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    logits = outputs.logits
    print(f"Training loss: {loss.item()}")

train_nlp_model()

# Step 4: Generate Policy Insights using OpenAI GPT
def generate_policy_recommendations(prompt):
    """Generate policy recommendations using OpenAI GPT."""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
    )
    return response.choices[0].message.content.strip()

policy_prompt = "Propose climate change mitigation policies based on rising global temperatures and CO2 levels."
policy_recommendations = generate_policy_recommendations(policy_prompt)
print("Policy Recommendations:")
print(policy_recommendations)

# Step 5: Combine with Geospatial Data Visualization
def visualize_geospatial_data(df):
    """Visualize climate data on a world map using Folium."""
    map_center = [20, 0]  # Approximate center of the world map
    world_map = folium.Map(location=map_center, zoom_start=2)

    # Example: Highlight countries based on temperature anomalies
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,
            color="red",
            fill=True,
            fill_color="red",
            tooltip=f"Temp: {row['temperature']:.2f}°C"
        ).add_to(world_map)

    # Save and display the map
    world_map.save("climate_map.html")
    print("Geospatial visualization saved as climate_map.html.")

# Example geospatial data
geospatial_data = pd.DataFrame({
    "latitude": [51.5, -33.9, 35.6],
    "longitude": [-0.1, 151.2, 139.7],
    "temperature": [15.5, 18.7, 20.3]
})

visualize_geospatial_data(geospatial_data)

# Step 6: Combine Multimodal Insights
def generate_multimodal_climate_report(climate_df, policy_text, map_file):
    """Generate a multimodal climate report."""
    report = f"""
    ## Climate Report

    ### Global Temperature Trends
    {climate_df.to_markdown()}

    ### Policy Recommendations
    {policy_text}

    ### Visualization
    See the attached geospatial map: {map_file}.
    """
    with open("climate_report.md", "w") as f:
        f.write(report)
    print("Multimodal climate report generated: climate_report.md")

generate_multimodal_climate_report(climate_data, policy_recommendations, "climate_map.html")


API clients initialized successfully.
Dataset URL: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data
Dataset downloaded to ./data.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training loss: 0.7010793089866638
Policy Recommendations:
1. Energy Efficiency and Conservation: The government should introduce new regulations that compel industries and homes to adopt energy-efficient practices. This includes using energy-saving appliances and the use of green energy technology. 

2. Carbon Pricing: Policymakers should consider establishing or enhancing programs that put a price on carbon emissions such as carbon taxes or cap-and-trade systems.

3. Renewable Energy Transition: Governments must invest in renewable energy technologies like solar, wind, and hydroelectric power to lower reliance on fossil fuels and thus lower CO2 emissions. 

4. Forestation and Reforestation: Develop forest management policies to increase carbon absorption capacity, incentivize the planting of trees, and make policies against deforestation more stringent.

5. Agriculture Management: Reform agricultural practices to enhance soil carbon sequestration, manage livestock GHG emissions, and t